In [ ]:
# Mount Google Drive
from google.colab import drive
import zipfile
import os

# Mount your Google Drive
drive.mount('/content/drive')

# Navigate to your zip file and extract it
zip_path = '/content/drive/MyDrive/your_rip_dataset.zip'  # Update with your actual path
extract_path = '/content/rip_dataset'

# Extract the zip file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

# Verify the structure
for root, dirs, files in os.walk(extract_path):
    level = root.replace(extract_path, '').count(os.sep)
    indent = ' ' * 2 * level
    print(f'{indent}{os.path.basename(root)}/')
    subindent = ' ' * 2 * (level + 1)
    for file in files[:5]:  # Show first 5 files only
        print(f'{subindent}{file}')
    if len(files) > 5:
        print(f'{subindent}... and {len(files)-5} more files')